In [10]:
# import necessary libraries
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable

In [11]:
# Define the data of the seats:
empty = pd.DataFrame({'IsAisle':[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                      'HasWindow': [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                      'IsNextTo':[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
                      'IsFourFacing':[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
                      'IsFourFree':[1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
                      'NrSurrounding_1':[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 1],
                      'NrFromDoor':[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 3, 3, 2, 2, 1, 1, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 3, 3, 2, 2, 1, 1, 0, 0],
                      'Available':[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]})

In [12]:
# Read the data
raw_data = pd.read_excel('data_discrete_choice_model.xlsx')
display(raw_data)

,ID,Trip Frequency,Age,Gender,Seat sit 1,Seat sit 2,Seat sit 3
0,1,1-3 times per week,25-34,Male,8,30,4
1,2,1-3 times per week,35-44,Female,32,37,39
2,3,More than 4 times per week,25-34,Female,19,14,14
3,4,1-3 times per week,18-24,Male,32,16,20
4,5,Once a month,25-34,Male,10,23,39
...,...,...,...,...,...,...,...
508,509,1-3 times per week,18-24,Female,9,15,21
509,510,1-3 times per week,35-44,Male,9,12,10
510,511,More than 4 times per week,25-34,Female,30,12,17
511,512,1-3 times per week,25-34,Male,34,11,12


In [13]:
person_data = pd.DataFrame(np.zeros(shape=(len(raw_data), 8)), 
                           columns=['IsFrequent', 'IsInfrequent', 'IsYoung', 'IsOld', 'IsMale', 'Empty', 'Alone', 'Busy'])

for i in range(len(raw_data)):
    if raw_data['Trip Frequency'][i] == 'More than 4 times per week':
        person_data['IsFrequent'][i] = 1
    
    if raw_data['Trip Frequency'][i] == 'Once a month':
        person_data['IsInfrequent'][i] = 1
    
    mean_age = np.mean([int(raw_data['Age'][i][:2]), int(raw_data['Age'][i][-2:])])
    if mean_age < 30:
        person_data['IsYoung'][i] = 1
    
    if mean_age > 65:
        person_data['IsOld'][i] = 1
    
    if raw_data['Gender'][i] == 'Male':
        person_data['IsMale'][i] = 1
    
    person_data['Empty'][i] = int(raw_data['Seat sit 1'][i])
    person_data['Alone'][i] = int(raw_data['Seat sit 2'][i])
    person_data['Busy'][i] = int(raw_data['Seat sit 3'][i])

person_data = person_data.drop(index=[192, 321, 330])
person_data = person_data.reset_index()

In [14]:
# Define variables
database = db.Database('persons', person_data)

FREQUENT = Variable('IsFrequent')
INFREQUENT = Variable('IsInfrequent')
YOUNG = Variable('IsYoung')
OLD = Variable('IsOld')
MALE = Variable('IsMale')
CHOICE = Variable('Empty')

In [15]:
# Parameters to be estimated
B_AISLE =Beta('B_AISLE', 0, None, None, 0)
B_WINDOW =Beta('B_WINDOW', 0, None, None, 0)
B_NEXTO =Beta('B_NEXTTO', 0, None, None, 0)
B_FOURFACING =Beta('B_FOURFACING', 0, None, None, 0)
B_FOURFREE =Beta('B_FOURFREE', 0, None, None, 0)
B_NRSUR =Beta('B_NRSUR', 0, None, None, 0)
B_NRDOOR =Beta('B_NRDOOR', 0, None, None, 0)
B_FREQ =Beta('B_FREQ', 0, None, None, 0)
B_INFREQ =Beta('B_INFREQ', 0, None, None, 0)
B_YOUNG =Beta('B_YOUNG', 0, None, None, 0)
B_OLD =Beta('B_OLD', 0, None, None, 0)
B_MALE =Beta('B_MALE', 0, None, None, 0)

In [17]:
# Definition of the utility functions
funcs = {}
for i in range(37):
    if empty['Available'][i] != 0:
        funcs["V{0}".format(i+1)] = B_FREQ * FREQUENT
        funcs["V{0}".format(i+1)] += B_INFREQ * INFREQUENT
        funcs["V{0}".format(i+1)] += B_YOUNG * YOUNG
        funcs["V{0}".format(i+1)] += B_OLD * OLD
        funcs["V{0}".format(i+1)] += B_MALE * MALE

        if empty['IsAisle'][i] == 1:
            funcs["V{0}".format(i+1)] += B_AISLE
        
        if empty['HasWindow'][i] == 1:
            funcs["V{0}".format(i+1)] += B_WINDOW

        if empty['IsNextTo'][i] == 1:
            funcs["V{0}".format(i+1)] += B_NEXTO

        if empty['IsFourFacing'][i] == 1:
            funcs["V{0}".format(i+1)] += B_FOURFACING

        if empty['IsFourFree'][i] == 1:
            funcs["V{0}".format(i+1)] += B_FOURFREE

        # if empty['NrSurrounding_1'][i] != 0:
        #     funcs["V{0}".format(i+1)] = B_NRSUR * empty['NrSurrounding_1'][i]

        # if empty['NrFromDoor'][i] != 1:
        #     funcs["V{0}".format(i+1)] = B_NRDOOR * empty['NrFromDoor'][i]


if empty['IsFourFacing'][i] == 1:
    funcs["V{0}".format(38)] = B_FOURFACING

if empty['IsFourFree'][i] == 1:
    funcs["V{0}".format(38)] += B_FOURFREE

nr_funcs = len(funcs)
print(nr_funcs)
print(funcs)

35
{'V1': ((((((((B_FREQ(init=0) * IsFrequent) + (B_INFREQ(init=0) * IsInfrequent)) + (B_YOUNG(init=0) * IsYoung)) + (B_OLD(init=0) * IsOld)) + (B_MALE(init=0) * IsMale)) + B_WINDOW(init=0)) + B_NEXTTO(init=0)) + B_FOURFREE(init=0)), 'V3': ((((((B_FREQ(init=0) * IsFrequent) + (B_INFREQ(init=0) * IsInfrequent)) + (B_YOUNG(init=0) * IsYoung)) + (B_OLD(init=0) * IsOld)) + (B_MALE(init=0) * IsMale)) + B_FOURFREE(init=0)), 'V4': (((((((B_FREQ(init=0) * IsFrequent) + (B_INFREQ(init=0) * IsInfrequent)) + (B_YOUNG(init=0) * IsYoung)) + (B_OLD(init=0) * IsOld)) + (B_MALE(init=0) * IsMale)) + B_AISLE(init=0)) + B_FOURFACING(init=0)), 'V5': ((((((B_FREQ(init=0) * IsFrequent) + (B_INFREQ(init=0) * IsInfrequent)) + (B_YOUNG(init=0) * IsYoung)) + (B_OLD(init=0) * IsOld)) + (B_MALE(init=0) * IsMale)) + B_FOURFREE(init=0)), 'V6': ((((((((B_FREQ(init=0) * IsFrequent) + (B_INFREQ(init=0) * IsInfrequent)) + (B_YOUNG(init=0) * IsYoung)) + (B_OLD(init=0) * IsOld)) + (B_MALE(init=0) * IsMale)) + B_AISLE(ini

In [18]:
# Associate utility functions with the numbering of alternatives
V = {}
for i in range(1, 40, 1):
    if empty['Available'][i-1] != 0:
        V[i] = funcs["V{0}".format(i)]

# Associate the availability conditions with the alternatives
av = {}
for i in range(1, 40 ,1):
    if empty['Available'][i-1] != 0:
        av[i] = 1

# Definition of the model
logprob = models.loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '01Empty'

# Calculate the null log likelihood for reporting
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

                 Value  Rob. Std err  Rob. t-test  Rob. p-value
B_AISLE      -0.899345      0.101133    -8.892691  0.000000e+00
B_FOURFACING -3.772584      0.711915    -5.299208  1.163062e-07
B_FOURFREE   -0.684324      0.085485    -8.005174  1.110223e-15
B_FREQ        3.735042      0.718247     5.200217  1.990556e-07
B_INFREQ      4.254569      0.698834     6.088095  1.142621e-09
B_MALE        4.732162      0.687054     6.887613  5.673684e-12
B_NEXTTO     -3.531505      0.708236    -4.986338  6.153444e-07
B_OLD         3.301538      0.732778     4.505509  6.621393e-06
B_WINDOW      0.822410      0.086166     9.544524  0.000000e+00
B_YOUNG       4.445489      0.722454     6.153314  7.588046e-10
